<table>
   <tr>     
    <td><img src="./images/logo-qiskit.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="200 px" align="left"></td>
    <td>  </td>
    <td><img src="./images/logo-IBM.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="200 px" align="left"></td>
   </tr>
</table>
<br>


# <center>Introduction à Qiskit</center>




Ces exercices ont pour but de vous familiariser avec les éléments de base de qiskit et des circuits quantiques. 

Commençons par définir ce qu'est un circuit quantique : 

> **"Un circuit quantique est une routine computationnelle d'opérations cohérentes sur des données quantiques, comme des états de qubits. Il s'agit d'une séquence ordonnée de portes quantiques, de mesures d'états, de remise à zéro, qui peut être conditionnée par du  calcul classique en temps réel."** (https://qiskit.org/textbook/ch-algorithms/defining-quantum-circuits.html)

Si cette défiinition peut parler à des physiciens, elle peut vous sembler obscure, cese exercices vont vous permettre de manipuler ett mesurer les états des qubits, en leur  appliquant des portes quantiques. Vous serz alors en mesure de créer vos propres circuitts quantiques. 

Avant de commencer, il faut executer la première cellule ci-dessous en la selectionnant et en appuyant sur `shift + enter`. C'est la manière standard de d'executer du code dans l'environement des notebooks Jupyter. Pendant l'execution, vous verrez `In [*]:` en hauut à gauche de la cellule. Lorsque l'exécution est terminée, vouos verrer un nombre à la place de l'étoile, ce nombre vous indique le nombre d'executions de celulles qui on été faites. Plus d'informations sur les notebooks Jupyter ici :  https://qiskit.org/textbook/ch-prerequisites/python-and-jupyter-notebooks.html.

La difficulté des exercices est indiquée par les couleurs de leur titres comme ceci :<span style="color:green"><em> facile </span><span style="color:blue"> moyen <span style="color:red"> difficile </em></span>


In [ ]:
import numpy as np

from qiskit import Aer, QuantumCircuit, execute
from qiskit.visualization import plot_histogram, plot_state_qsphere
from qiskit.quantum_info import Statevector
from IPython.display import display, Math, Latex


##  Opérations de base sur les qubits, notions préliminaires 

### Ecriture des états de qubits 
Commençons par un seul qubit. A la différence d'un bit classique qui ne peut prendre que les valeurs 0 ou 1, un quantum bit ou **qubit**, peut être dans les états $\vert0\rangle$, $\vert1\rangle$, ainsi que dans une combinaison linéraire de ces deux états. C'est ce que l'on appelle la superposition, ce qui nous permet d'écrire l'état général d'un qubit sous cette forme : 

$$\vert\psi\rangle = \sqrt{1-p}\vert0\rangle + e^{i \phi} \sqrt{p}  \vert1\rangle$$

Et si nous devions mesurer l'état de ce qubit, nous trouverions le resultat $1$ avec la probabilité $p$, et le resultat $0$ avec la probabilité $1-p$. Comme on peut le vois la somme des probabilités est $1$, c'eest à dire ue l'on va en effet mesurer soit $0$ soit $1$, et il n'y a pas d'autre résultats possibles.

En plus de $p$, il y a un autre paramètre : la variable $\phi$ indique la phase relative entre les deux états $\vert0\rangle$ et $\vert1\rangle$. Comme on va le voir, cette phase est importante, elle permet d'utiliser le phénomène d'interférence.

Pour en savoir plus vous pouvez consulter https://qiskit.org/textbook/ch-states/representing-qubit-states.html

### Visualisation des états quantiques
Dans cet exercice, nous utiliserons la `qsphere`. Voici la représentation de la  `qsphere` pour lees états  $\vert0\rangle$ et $\vert1\rangle$, respectivement. Notons que le haut de la sphère represente l'état $\vert0\rangle$, alors que le bas représente l'état $\vert1\rangle$.

<img src="./images/qsphere01.png" alt="qsphere with states 0 and 1" style="width: 400px;"/>

Alors il ne sera pas surprenant qu'un état de superposition de probabilité $p = 1/2$ (c'est à dire autant de chances de mesurer 0 que de chances de mesurer 1) apparaitra sur la  `qsphere` avec deux points. Noteze que la taille des cercles à ces deux points et plus petite que lorsque l'on a un seul point comme pour les états $\vert0\rangle$ ou  $\vert1\rangle$. C'est parceque les cercles ssont représentés avec des tailles proportionnelles aux probabilité de mesurer les états auxquels ils correspoondent. 

<img src="./images/qsphereplus.png" alt="qsphere with superposition 1" style="width: 200px;"/>

Si maintenant les états de superpositions pour lesquels la phase n'est pas nulle, la `qsphere` nous permet de visualiser la phase d'un état en changeant la couleur de la boule correspondante.  Par exemple, l'état pour lequel  $\phi = 90^\circ$ (degrees) et  $p = 1/2$ est montré sur la `qsphere` ci-dessous. 

<img src="./images/qspherey.png" alt="qsphere with superposition 2" style="width: 200px;"/>


### Manipulation des qubits
On fait évoluer el'état des qubits en appliquant des portes quantiques. Voici les définitions de quelques portes quantiques que nous utiliserons dans ce noteboook.

Premièrement, décrivons comment la valeur de $p$ change pour notre état quantique général, avec ces deux portes :

1. **$X$-gate**: Cette porte échange les états $\vert0\rangle$ et $\vert1\rangle$. Cette opération ressemble à la porte classique NON (NOT). Ainsi, la $X$-gate est parfois appelée bit flip ou NOT gate. Mathematiquement, la porte $X$ change $p$ en $1-p$, et donc en particulier 0 en 1 et vice versa. 

2. **$H$-gate**: Cette porte fait passer de l'état $\vert0\rangle$ à l'état $\frac{1}{\sqrt{2}}\left(\vert0\rangle + \vert1\rangle\right)$. Cet état est aussi appelé  $\vert+\rangle$. Mathematiquement, cela fait aller de $p=0, \phi=0$vers  $p=1/2, \phi=0$.  Comme l'état résultant est une superposton des états $\vert0\rangle$ et $\vert1\rangle$, la porte de Hadamard represente réellement une opération de nature quantique.

Notons que ces deux portes modifient la valeur de $p$, mais pas celle de $\phi$. 

Plus généralement, le dessin ci-dessous nous permet de visualiser les actions de ces portes : 

<img src="./images/quantumgates.png" alt="quantum gates" style="width: 400px;"/>

Depuis l'état $\vert+\rangle$, on peut changer la phase en appliquant d'autres porte. Par exemple la porte $S$ gate ajoute $90$ degrés à $\phi$, tandis que la porte $Z$ ajoute $180$ degrés à $\phi$. Pour oter $90$ degrés à la phase, on peut utiliser la porte  $S^\dagger$, que l'on lit S-dagger, et que l'on peut écrire `sdg`. Enfin, il y a la porte $Y$ qui applique une séquence de portes $Z$ et $X$ gates.

Pour een savoir plus à propos de la description des états quantiques, des opérateurs de Pauli et des autres portes à un qubit, vous pouvez vous réfereer au premier chapitre du textbook : https://qiskit.org/textbook/ch-states/introduction.html.

Voici quelques exercices pour obtenir les différents états et les visualiser sur le `qsphere`. Voici la syntaxe à utiliser pour utiliser les différentes portes :

    qc.x(0)    # bit flip
    qc.y(0)    # bit flip et phase flip
    qc.z(0)    # phase flip
    qc.h(0)    # superpostion
    qc.s(0)    # rotation de phase de pi/2 (90 degrees)
    qc.sdg(0)  # rotation de phase de -pi/2 (90 degrees)
    
Le '(0)' indique que nous appliquons la porte au qubit 'q0', qui est le premier (et dans ce cas, le seul) qubit.

Dans les prochaines celulles, et afin de pouvoir visualiser l'état quantique d'un qubit, nous n'utilisons pas le simulateur usuel (`qasm_simulator`) pour lequel les qubits démarrent tous à 0, un circuit est appliqué et des mesures sont faites pour produire dse résultats. 

Au lieu de cela nous utiliserons le `statevector_simulator`, avec la possibilité de créer et composer un circuit, de démarrer depuis un état que l'on peut désigner (`Statevector.from_label('label)` )et de faire évoluer (`sv.evolvel(qc)` : `sv` étant un vecteur d'état et `qc` un circuit quantique) l'état pour visualiser l'état résultant à la sorti (sans le mesurer) par la fonction `plot_state_qshpere`.

Voici un exemple

In [ ]:
# Exemple
def create_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(n): 
        qc.h(i)
    return qc


n = 4
qc = create_circuit(n)

sv = Statevector.from_label('0'*n)
sv = sv.evolve(qc)

plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

A présent, revenons à 1 qubit, à vous de jouer

## <span style="color:green"><em>1. Opérations de base</em></span>

### <span style="color:green"><em>1.1 Commençons par un bit-flip, le but est d'atteindre l'état  $\vert1\rangle$ en partant de l'état $\vert0\rangle$. </em></span>

On veut atteindre l'état : 

<img src="./images/state1.png" width="300"> 

(nb: il se peut qu'au fil des versions, le choix de couleur pour la phase 0 ait changé)

In [ ]:
# 1.1
def create_circuit():
    qc = QuantumCircuit(1)
    # Composez votre circuit ici
    
    
    
    return qc

# verifier

qc = create_circuit()

sv = Statevector.from_label('0')
sv = sv.evolve(qc)

plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

### <span style="color:green"><em>1.2 Créons un état superposé, le but est d'atteindre l'état  $|+\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle + |1\rangle\right)$ en partant de l'état $\vert0\rangle$. </em></span>
<img src="./images/stateplus.png" width="300"> 



In [ ]:
# 1.2
def create_circuit():
    qc = QuantumCircuit(1)
    # Composez votre circuit ici
    
    
    #
    return qc

qc = create_circuit()
sv = Statevector.from_label('0')
sv = sv.evolve(qc)
plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

### <span style="color:green"><em>1.3 En combinant les 2 précédants, nous pouvons attendre  l'état  $|+\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle - |1\rangle\right)$  partant de l'état $\vert0\rangle$. </em></span>
<img src="./images/stateminus.png" width="300"> 

In [ ]:
# 1.3
def create_circuit():
    qc = QuantumCircuit(1)
    # Composez votre circuit ici #####
   
   
    
    ##################################
    return qc

qc = create_circuit()
sv = Statevector.from_label('0')
sv = sv.evolve(qc)
plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

### <span style="color:green"><em>1.4 Finalement, voyons un état faisant intervenir dse coordonnéeese complexes, tentons d'atteindre l'état  $|\circlearrowleft\rangle = \frac{1}{\sqrt{2}}\left(|0\rangle - i|1\rangle\right)$ en partant de l'état $\vert0\rangle$. </em></span>
    
 <img src="./images/stateleft.png" width="300"> 


In [ ]:
# 1.4
def create_circuit():
    qc = QuantumCircuit(1)
    # Composez votre circuit ici #######
    
    
    ####################################
    return qc

qc = create_circuit()
sv = Statevector.from_label('0')
sv = sv.evolve(qc)
plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 


## 2. Circuits quantiques utilisant des portes à plusieurs qubits

Les portes à deux qubits usuelles ont pour syntaxe :

    qc.cx(c,t)       # controlled-X (= CNOT) c est le qubit de contrôle et t est le qubit 
                     # cible (target) 
    qc.cz(c,t)       # controlled-Z c est le qubit de contrôle et t est le qubit 
                     # cible (target) 
    qc.swap(a,b)     # SWAP porte qui échange les états des qubits a et b 
    
Pour plus de détails au sujet des portes à plusieurs qubits : https://qiskit.org/textbook/ch-gates/introduction.html.

We can now have up to four points on the qsphere.

Nous démarrons avec la porte à deux qubit : Controlled-NOT (ou CNOT ou CX, ou encore $C_x$. Comme dans toutes les portes à contrôle, un qubit est désigné controleur et l'autre cible (target). 
Si le qubit de contrôle est dans l'état $|0\rangle$, alors l'état du qubit cible reste inchangé (l'opérateur Identité est appliqué), rien ne se passe. Si le qubit de contrôle est dans l'état $|1\rangle$, alors une porte X est appliquée au qubit cible.

Ainsi lorsque les deux qubits sont dans les états $|0\rangle$ ou $|1\rangle$, la porte CNOT effectue des opérations "classiques"

Mais la situation change complètement si on applique d'abord une porte de Hadamard au qubit de contrôle, et qu'il se trouve dans l'état $|+\rangle$. L'action de la porte CNOT sur cette entrée non classique produit des états intriqués entre le qubit de contrôle et le qubit cible. 

Si le qubit cible est initialenent dans l'état $|0\rangle$, alors l'état résultant est noté $|\Phi^+\rangle$, et il s'agit d'un des états appelés états de Bell. 


### <span style="color:green"><em>2.1 Contruisons l'état de Bell $|\Phi^+\rangle = \frac{1}{\sqrt{2}}\left(|00\rangle + |11\rangle\right)$. </em></span> 

<img src="./images/phi+.png" width="300"> 
Dans cet état, la probabilité de mesurer "00" vaut $\frac{1}{2}$ de même que la probabilité de mesurer "11", ainsi les résultats des mesures de qubits sont parfaitement correlés.

In [ ]:
# 2.1 
def create_circuit():
    qc = QuantumCircuit(2)
    # Composez votre circuit ici #####
    


    ##################################
    return qc

qc = create_circuit()
sv = Statevector.from_label('00')
sv = sv.evolve(qc)
plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

 ###  <span style="color:green"><em>2.2 Maintenant construisons un autre état de Bell :  $\vert\Psi^-\rangle = \frac{1}{\sqrt{2}}\left(\vert01\rangle - \vert10\rangle\right)$. <img src="./images/psi-.png" width="300"> 

Ici, les qubits sont anti-correlés. Notez aussi le signe moins, qui indique la phase relaive entre les deux états.

In [ ]:
# 2.2 
def create_circuit():
    qc = QuantumCircuit(2,2) # this time, we not only want two qubits, but also 
                             # two classical bits for the measurement later
        
    # Composez votre circuit ici #####
    
    ##################################
    
    return qc

qc = create_circuit()
sv = Statevector.from_label('00')
sv = sv.evolve(qc)
print(sv)
plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

Ci-dessous, nous avons ajouté des opérations de mesure au circuit. Pour recueillir le résultat de la mesure, nous avons besoin de bits classiques, c'est pouruqoi le circuit a été créé avec deux arguments comme ceci : `qc = QuantumCircuit(num_qubits, num_classicalbits)`.

Puis nous définissons une fonction `run_circuit()` pour exécuter un circuit (passé en paramètre) sur le simulateur local. Si l'état a été correctement préparé on aura la probabilité $\frac{1}{2}$ de mesurer chacun des deux résultats possibles "01" et "10". Pour autant, le résultat de la mesure pour 1000 expériences ne signifie par que nous mesureons "00" exactement 500 fois (un peu comme lorsque l'on tire à pile ou face, il est peu probable de trouver **exactement**   un partage à 50/50.  Il y a en fait des fluctuations autour de cette valeur. On peut lancer `run_circuit` à plusieurs reprises pour observer ce phénomène.

In [ ]:
qc.measure(0, 0) # mesure du qubit q_0 et stockage du résultat dans le bit classique c_0
qc.measure(1, 1) # mesure du qubit q_1 et stockage du résultat dans le bit classique c_1
qc.draw(output='mpl') # on "dessine" le circtui

In [ ]:
def run_circuit(qc):
    backend = Aer.get_backend('qasm_simulator') # on choisit le simulateur basique comme backend
    result = execute(qc, backend, shots = 1000).result() # on lance la simulation
    counts = result.get_counts() # on récupère les comptages
    return counts

counts = run_circuit(qc)
print(counts)
plot_histogram(counts) # affichons les résultats sous forme d'histogramme

### <span style="color:blue"><em>2.3 on donne le circuit suivant. </em></span>
    
Swappez les états entre les deux qubits.  
This should be your final state: <img src="./images/stateIIiii.png" width="300"> 

In [ ]:
# 9
def create_circuit():
    qc = QuantumCircuit(2)
    ## partie initialisation
    qc.rx(np.pi/3,0)
    qc.x(1)
    ## fin de l'initialisation 
    # Composez votre circuit ici #####
    
    
    #################################
    
    return qc


qc = create_circuit()
sv = Statevector.from_label('00')
sv = sv.evolve(qc)
print(sv)
plot_state_qsphere(sv.data, show_state_labels=True, show_state_phases=True) 

### <span style="color:blue"><em>2.4 Ecrivez un programme depuis le début qui produit l'état GHZ sur 3 qubit$\vert \text{GHZ}\rangle = \frac{1}{\sqrt{2}} \left(|000\rangle + |111 \rangle \right)$, effetcuez les mesures sur 2000 occurences, affichez le résultat. </em></span>
    
<img src="./images/ghz.png" width="300"> 


In [ ]:
# Composez votre code ici #####

#


In [ ]:
# Composez votre code ici #####

#

# <span style="color:red"><em>3.1 : L'additionneur : </em></span>


- Le dessin ci-dessous représente le circuit pour un additionneur à deux qubit avec retenue. 
<img src="./images/adder.png" alt="full 2 qubit adder" style="width: 600px;"/>

- Construire le circuit quantique correspondant,

- Automatisez le pour vérifier le résultat pour les 8 entrées possibles (000 à 111),

- Que se passe-t'il si on lui fournit (en A,B,$C_{in}$) une superposition des 3 valeurs ? 


In [ ]:
# Composez votre code ici #####


#

# <span style="color:red"><em>4.1 : Superposez vos initiales : </em></span>


Le but de cet exercice est de construire un circuit qui produit deux bitstrings différents, de manière équiprobable, en utilisant la superposition et l'intrication.


On utilise un registre quantique de longueur 7 et le code ASCII pour représenter les lettres de A à Z. (de $b'1000001'$ pour le A à $b'1011010'$ pour le Z Z). 

Choisissez 2 lettres de l'aphabet, ce peut-être vos initiales. Par exemple Thomas Gaurond choisirait G  et T.

Et le résultat ressemplberait à ceci 

<img src="./images/GT-initials.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="350 px" align="center">


Les import Python, et l'outil pour afficher les lettre sont fournis, il ne vous reste qu'à construire le circuit.

Il vous faut donc :

- choisir deux lettres (différentes)
- trouver leur code en ASCII
- construire un circuit quantique qui produit en sortie la superposition de ces deux valeurs 

Voici la table des codes ASCII de A à Z : 


| Letter | binary ASCII value | Letter | binary ASCII value | 
| ------ | ------------------ | ------ | ------------------ |
| A | 100 0001 | N | 100 1110 |
| B | 100 0010 | O | 100 1111 | 
| C | 100 0011 | P | 101 0000 |
| D | 100 0100 | Q | 101 0001 |
| E | 100 0101 | R | 101 0010 |
| F | 100 0110 | S | 101 0011 |
| G | 100 0111 | T | 101 0100 |
| H | 100 1000 | U | 101 0101 |
| I | 100 1001 | V | 101 0110 |
| J | 100 1010 | W | 101 0111 |
| K | 100 1011 | X | 101 1000 |
| L | 100 1100 | Y | 101 1001 |
| M | 100 1101 | Z | 101 1010 |


In [ ]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute
from qiskit.tools.visualization import plot_histogram
from qiskit import IBMQ, BasicAer
from qiskit.tools.jupyter import *
import matplotlib.pyplot as plt
%matplotlib inline

### Construisez votre circuit ci-dessous, en prenant soin de le nommer `qc`


In [ ]:
# ici

In [ ]:
### NE PAS MODIFIER LE CODE CI-DESSOUS
### il va executer vottre circuit
### et produire l'affichage de la superposition des lettres :

backend = BasicAer.get_backend('qasm_simulator')
shots_sim = 10

job_sim = execute(qc, backend, shots=shots_sim)
stats_sim = job_sim.result().get_counts()

def plot_letter (stats, shots):
    ### code from the qiskit.org smiley demo
    for bitString in stats:
        char = chr(int( bitString[0:7] ,2)) 
        prob = stats[bitString] / shots 
      
        plt.annotate( char, (0.5,0.5), va="center", ha="center", color = (0,0,0, prob ), size = 300)
        if (prob>0.05): 
            print(str(prob)+"\t"+char)
    plt.axis('off')
    plt.show()
    
print(shots_sim) 
print(stats_sim)
plot_letter(stats_sim, shots_sim)